## Reference

After running a lot of my own code, I copied and transformed the code of https://www.kaggle.com/viveksrinivasan 

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_squared_log_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer,FunctionTransformer,MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.pipeline import make_pipeline,Pipeline
import math
from sklearn.ensemble import RandomForestClassifier,GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report 

In [3]:
cd ..

/Users/oguzhanbekar/OneDrive/Spiced/euclidean-eukalyptus-student-code/03_week


In [4]:
df = pd.read_csv('data/train.csv',index_col='datetime',parse_dates=True) 

# Feature Engineering before splitting

In [5]:
df = df[np.abs(df["count"]-df["count"].mean())<=(3*df["count"].std())] 

In [6]:
X = df
#.iloc[:,:-3]
y = df['count']

In [7]:
# splitting time
X['hour'] = X.index.hour
X['day'] = X.index.day
X['month'] = X.index.month
X['year'] = X.index.year
X['weekday'] = X.index.dayofweek

In [8]:
categoricalFeatureNames = ["season","holiday","workingday","weather","weekday","month","year","hour"]
numericalFeatureNames = ["temp","humidity","windspeed"]
# dropFeatures = ['casual',"count","datetime","wdate","registered"]

In [9]:
for var in categoricalFeatureNames:
    X[var] = X[var].astype("category")

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10739 entries, 2011-01-01 00:00:00 to 2012-12-19 23:00:00
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   season      10739 non-null  category
 1   holiday     10739 non-null  category
 2   workingday  10739 non-null  category
 3   weather     10739 non-null  category
 4   temp        10739 non-null  float64 
 5   atemp       10739 non-null  float64 
 6   humidity    10739 non-null  int64   
 7   windspeed   10739 non-null  float64 
 8   casual      10739 non-null  int64   
 9   registered  10739 non-null  int64   
 10  count       10739 non-null  int64   
 11  hour        10739 non-null  category
 12  day         10739 non-null  int64   
 13  month       10739 non-null  category
 14  year        10739 non-null  category
 15  weekday     10739 non-null  category
dtypes: category(8), float64(3), int64(5)
memory usage: 841.2 KB


# Trying

## Train Test Split

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3)

In [12]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((7517, 16), (7517,), (3222, 16), (3222,))

In [13]:
# dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
# dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])
# datetimecol = dataTest["datetime"]
# yLabels = dataTrain["count"]
# yLablesRegistered = dataTrain["registered"]
# yLablesCasual = dataTrain["casual"]

## RMSLE Function

In [14]:
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

## Feature Engineering

In [15]:
categorical_features = ['season','weather','hour','day','month','year','weekday']
categorical_transformer = OneHotEncoder(sparse=False,handle_unknown='ignore')

polyScaleFeatures = ['temp','windspeed','humidity']
poly2Features = ['month','weather','workingday','year','hour','temp','humidity','weekday']


In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ploy and scale',make_pipeline(PolynomialFeatures(degree=5,include_bias=False),
                                        MinMaxScaler()),polyScaleFeatures),
        ('ploy2', PolynomialFeatures(interaction_only=True,include_bias=False),poly2Features),
        ("cat", categorical_transformer, categorical_features),
        ('do nothing','drop',['holiday','atemp','casual','count','registered'])
    ],
    remainder='passthrough'
)


1. alpha (gradient descent learning rate)
2. Regularization coefficient (how strong the regularization is) and the regularization type (L1, L2, and hybrid)
3. stopping criterion for the gradient descent
4. Max of number iteration
5. the solver (the type/flavor of gradient descent algorithm)

In [17]:
# hyperparam_grid = {
#     'learningr': [], 
#     'n_estimators': [5, 10, 100, 200],
#     'min_samples_leaf': [1,5,10]
# }

# grid_cv = GridSearchCV(estimator=LinearRegression(),            # unfitted model/estimator
#                        param_grid=hyperparam_grid,  # hyperparameters dict
#                        cv=5,                        # number of folds, k
#                        scoring='accuracy')          # scoring metric

In [18]:
# grid_cv.fit(X_train, y_train)

In [ ]:
X_train.head()

## Using all Training Data 

In [21]:
dataTrain = pd.read_csv("data/train.csv",index_col='datetime',parse_dates=True)
dataTest = pd.read_csv("data/test.csv",index_col='datetime',parse_dates=True)

In [22]:
data = dataTrain.append(dataTest)

In [23]:
data['hour'] = data.index.hour
data['day'] = data.index.day
data['month'] = data.index.month
data['year'] = data.index.year
data['weekday'] = data.index.dayofweek

In [24]:
categoricalFeatureNames = ["season","holiday","workingday","weather","weekday","month","year","hour"]
numericalFeatureNames = ["temp","humidity","windspeed","atemp"]
dropFeatures = ['casual',"count","datetime","registered",'day']

In [25]:
for var in categoricalFeatureNames:
    data[var] = data[var].astype("category")

In [26]:
data.reset_index(inplace=True)
# data.drop('index',inplace=True,axis=1)

In [27]:
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month,year,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0,0,1,1,2011,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0,1,1,1,2011,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0,2,1,1,2011,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0,3,1,1,2011,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0,4,1,1,2011,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN,19,31,12,2012,0
17375,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN,20,31,12,2012,0
17376,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,NaN,NaN,21,31,12,2012,0
17377,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,NaN,NaN,22,31,12,2012,0


In [28]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])
datetimecol = dataTest["datetime"]
yLabels = dataTrain["count"]
yLablesRegistered = dataTrain["registered"]
yLablesCasual = dataTrain["casual"]

In [29]:
dataTest.set_index('datetime',inplace=True)

In [30]:
dataTrain  = dataTrain.drop(dropFeatures,axis=1)
dataTest  = dataTest.drop(dropFeatures,axis=1)

KeyError: "['datetime'] not found in axis"

In [31]:
dataTest.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month,year,weekday
datetime,,,,,,,,,,,,,,,,
2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,NaN,NaN,NaN,0,20,1,2011,3
2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,NaN,NaN,NaN,1,20,1,2011,3
2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,NaN,NaN,NaN,2,20,1,2011,3
2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,NaN,NaN,NaN,3,20,1,2011,3
2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,NaN,NaN,NaN,4,20,1,2011,3


## Training Data

In [32]:
pipeline = make_pipeline(preprocessor, LinearRegression())

In [33]:
X

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month,year,weekday
datetime,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,1,1,2011,5
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1,1,1,2011,5
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2,1,1,2011,5
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,3,1,1,2011,5
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,4,1,1,2011,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19,19,12,2012,2
2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20,19,12,2012,2
2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21,19,12,2012,2


In [34]:
pipeline.fit(X = X,y = y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ploy and scale',
                                                  Pipeline(steps=[('polynomialfeatures',
                                                                   PolynomialFeatures(degree=5,
                                                                                      include_bias=False)),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['temp', 'windspeed',
                                                   'humidity']),
                                                 ('ploy2',
                                                  PolynomialFeatures(include_bias=False,
                                                                     interaction_only=T

In [35]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])
datetimecol = dataTest["datetime"]
yLabels = dataTrain["count"]
yLablesRegistered = dataTrain["registered"]
yLablesCasual = dataTrain["casual"]

In [36]:
dataTrain

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month,year,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0,0,1,1,2011,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0,1,1,1,2011,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0,2,1,1,2011,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0,3,1,1,2011,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0,4,1,1,2011,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7.0,329.0,336.0,19,19,12,2012,2
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10.0,231.0,241.0,20,19,12,2012,2
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4.0,164.0,168.0,21,19,12,2012,2
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12.0,117.0,129.0,22,19,12,2012,2


In [37]:
dataTrain.set_index('datetime',inplace=True)

In [38]:
yLabelsLog = np.log1p(yLabels)
pipeline.fit(X = dataTrain,y = yLabelsLog)
preds = pipeline.predict(X=dataTrain)
print ("RMSLE Value For Linear Regression: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))


RMSLE Value For Linear Regression:  0.5137238335159505


In [40]:
gbm = GradientBoostingRegressor(n_estimators=4000,alpha=0.01)
yLabelsLog = np.log1p(yLabels)
gbm.fit(X = dataTrain,y = yLabelsLog)
preds = gbm.predict(X=dataTrain)
print ("RMSLE Value For Gradient Regression: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))

# gbm = GradientBoostingRegressor(n_estimators=4000,alpha=0.01); ### Test 0.41
# yLabelsLog = np.log1p(yLabels)
# gbm.fit(dataTrain,yLabelsLog)
# preds = gbm.predict(X= dataTrain)
# print ("RMSLE Value For Gradient Boost: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))

# lModel = LinearRegression()
# # Train the model
# yLabelsLog = np.log1p(yLabels)
# lModel.fit(X = dataTrain,y = yLabelsLog)
# # Make predictions
# preds = lModel.predict(X= dataTrain)
# print ("RMSLE Value For Linear Regression: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))

RMSLE Value For Linear Regression:  0.00023315576513539363


## Predicting Real Test data

In [41]:
# pipeline.predict(X_test)
# predicted = gbm.predict(dataTest)
predicted_pip = pipeline.predict(df_test)

# predicted[predicted<0] = 0.0
# msle = mean_squared_log_error(y_test, y_predicted)
# msle
# count

NameError: name 'df_test' is not defined

In [ ]:
predicted_pip

In [ ]:
dataTest.set_index('datetime',inplace=True)

In [ ]:
dataTest

In [ ]:
predsTest = pipeline.predict(X= dataTest)


In [ ]:
predicted[predicted>0]

In [ ]:
submission = pd.DataFrame({
        "datetime": dataTest.index,
        "count": [max(0, x) for x in np.exp(predsTest)]
    })

In [ ]:
submission.head(20)

In [ ]:
submission.to_csv('Submission.csv',index=False)

# Testing my original Model with the Pipeline and all Training Data Set

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_squared_log_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer,FunctionTransformer,MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.pipeline import make_pipeline,Pipeline
import math
from sklearn.ensemble import RandomForestClassifier,GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import GridSearchCV


In [3]:
cd ..

/Users/oguzhanbekar/OneDrive/Spiced/euclidean-eukalyptus-student-code/03_week


In [4]:
df = pd.read_csv('data/train.csv',index_col='datetime',parse_dates=True) 

## Feature Engineering before splitting

In [5]:
df = df[np.abs(df["count"]-df["count"].mean())<=(3*df["count"].std())] 

In [11]:
X = df.iloc[:,:-3]
#.iloc[:,:-3]
y = df['count']

In [13]:
# splitting time
X['hour'] = X.index.hour
X['day'] = X.index.day
X['month'] = X.index.month
X['year'] = X.index.year
X['weekday'] = X.index.dayofweek

In [15]:
categorical_features = ['season','weather','hour','day','month','year','weekday']
categorical_transformer = OneHotEncoder(sparse=False,handle_unknown='ignore')

polyScaleFeatures = ['temp','windspeed','humidity']
poly2Features = ['month','weather','workingday','year','hour','temp','humidity','weekday']


In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ploy and scale',make_pipeline(PolynomialFeatures(degree=5,include_bias=False),
                                        MinMaxScaler()),polyScaleFeatures),
        ('ploy2', PolynomialFeatures(interaction_only=True,include_bias=False),poly2Features),
        ("cat", categorical_transformer, categorical_features),
        ('do nothing','drop',['holiday','atemp'])
    ],
    remainder='passthrough'
)


In [69]:
pipeline = make_pipeline(preprocessor, GradientBoostingRegressor(n_estimators=100,alpha=0.01)) ### Test 0.41)
pipeline.fit(X = X,y = y)
pipeline_rf = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=100)) ### Test 0.41)
pipeline_rf.fit(X = X,y = y)
pipeline_l = make_pipeline(preprocessor, Lasso(max_iter=3000,alpha=0.1)) ### Test 0.41)
pipeline_l.fit(X = X,y = y)
pipeline_r = make_pipeline(preprocessor, Ridge(max_iter=3000,alpha=0.1)) ### Test 0.41)
pipeline_r.fit(X = X,y = y)


/Users/oguzhanbekar/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 42397455.42995171, tolerance: 29833.06188434677
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ploy and scale',
                                                  Pipeline(steps=[('polynomialfeatures',
                                                                   PolynomialFeatures(degree=5,
                                                                                      include_bias=False)),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['temp', 'windspeed',
                                                   'humidity']),
                                                 ('ploy2',
                                                  PolynomialFeatures(include_bias=False,
                                                                     interaction_only=T

In [39]:
pipeline.score(X,y)

0.9054161925904357

## Load Test Dataset

In [23]:
df_test = pd.read_csv('data/test.csv',index_col='datetime',parse_dates=True)

In [28]:
# splitting time
df_test['hour'] = df_test.index.hour
df_test['day'] = df_test.index.day
df_test['month'] = df_test.index.month
df_test['year'] = df_test.index.year
df_test['weekday'] = df_test.index.dayofweek

In [58]:
predictions_train = pipeline.predict(X)

In [70]:
predictions = pipeline.predict(df_test)
predictions_rf = pipeline_rf.predict(df_test)
predictions_l = pipeline_l.predict(df_test)
predictions_r = pipeline_r.predict(df_test)

In [51]:
predictions

array([ 10.90325959,   7.94727062,   0.25567849, ..., 199.03351581,
       150.00196605, 117.58892156])

In [55]:
submission

,datetime,count
0,2011-01-20 00:00:00,10.903260
1,2011-01-20 01:00:00,7.947271
2,2011-01-20 02:00:00,0.255678
3,2011-01-20 03:00:00,0.255678
4,2011-01-20 04:00:00,0.025065
...,...,...
6488,2012-12-31 19:00:00,342.734735
6489,2012-12-31 20:00:00,256.551182
6490,2012-12-31 21:00:00,199.033516
6491,2012-12-31 22:00:00,150.001966


In [71]:
submission = pd.DataFrame({
        "datetime": df_test.index,
        "count": [max(0, x) for x in predictions]
    })
submission_rf = pd.DataFrame({
        "datetime": df_test.index,
        "count": [max(0, x) for x in predictions_rf]
    })
submission_l = pd.DataFrame({
        "datetime": df_test.index,
        "count": [max(0, x) for x in predictions_l]
    })
submission_r = pd.DataFrame({
        "datetime": df_test.index,
        "count": [max(0, x) for x in predictions_r]
    })

In [75]:
submission_r.to_csv('Submission.csv',index=False)

# Finding out what fit and transform do

In [79]:
df['season']

datetime
2011-01-01 00:00:00    1
2011-01-01 01:00:00    1
2011-01-01 02:00:00    1
2011-01-01 03:00:00    1
2011-01-01 04:00:00    1
                      ..
2012-12-19 19:00:00    4
2012-12-19 20:00:00    4
2012-12-19 21:00:00    4
2012-12-19 22:00:00    4
2012-12-19 23:00:00    4
Name: season, Length: 10739, dtype: int64

In [100]:
onehot = OneHotEncoder()

In [101]:
categorical_features = df[['season']]

In [105]:
seasonencoder = onehot.fit(df[['season']])

In [108]:
seasonx = onehot.transform(categorical_features)

In [110]:
seasonx.info

AttributeError: info not found